In [1]:
from bs4 import BeautifulSoup
import asyncio
from aiohttp import ClientSession
import numpy as np
import pandas as pd
from functools import partial
from aiohttp import ClientConnectorError
import json
import re
from bs4 import NavigableString
from pprint import pprint
from datetime import datetime, timedelta
import os
from itertools import chain
import pickle

In [2]:
trails = pickle.load(open("./data/mondo_scraped/cleaned_int_eco.pkl", "rb"))

In [3]:
trails[:10]

['http://www.elmundo.es/economia/2016/12/14/58518533e5fdea775d8b46a0.html',
 'http://www.elmundo.es/economia/2014/11/13/546467c622601d3f5e8b4573.html',
 'http://www.elmundo.es/economia/2014/12/01/547c20f4268e3e12718b456b.html',
 'http://www.elmundo.es/economia/2017/02/20/58a978a3e5fdea15138b4598.html',
 'http://www.elmundo.es/comunidad-valenciana/castellon/2017/05/20/59205a0546163f39768b4579.html',
 'http://www.elmundo.es/economia/2014/06/14/539ca128268e3e476c8b4570.html',
 'http://www.elmundo.es/internacional/2014/09/08/540d726c268e3e20298b456e.html',
 'http://www.elmundo.es/economia/2014/06/26/53abfa99ca4741fd2f8b4572.html',
 'http://www.elmundo.es/economia/2015/10/20/5626352a46163fb9188b457f.html',
 'http://www.elmundo.es/economia/2015/03/18/55095bf522601d807f8b456b.html']

In [4]:
len(trails)

14350

In [5]:
async def get_html_data(trail, session):
    """Access El Mondo webpage"""
    url = trail
    attempts = 0
    while attempts < 3:
        try:
            res = await session.get(url)
            try:
                html = await res.text()
            except UnicodeDecodeError:
                return trail, None
            try:
                soup = BeautifulSoup(html, 'html.parser')
                headline = soup.select("h1[itemprop='headline']")[0].text
                facts = [fact.text for fact in soup.select("p.summary-lead, h2[itemprop='alternativeHeadline']")]
                body = "\n".join([sent.text for sent in soup.select("div[itemprop='articleBody']")[0]
                                 .find_all('p', class_=lambda x: x != 'summary-lead')])
                tag = [tag.text for tag in soup.select("ul.item-tags a")]
            except (IndexError, TypeError) as e:
                global DEBUG
                DEBUG.append((trail, e, soup))
                print("content error!")
                return trail, None
            try:
                time = soup.select("time")[0]['datetime']
            except:
                time = None
            attrs = {
                "headline": headline,
                "keyfacts": facts, 
                "content": body,
                "tags": tag,
                "time": time
                    }
            return trail, attrs
        except ClientConnectorError:
            attempts += 1
            print("Connector error occurred!")
    if attempts == 3:
        return trail, None
    
async def gather_results(curr, step, trails):
    """Launch scrape tasks and collect results"""
    tasks = []
    async with ClientSession() as session:
        for trail in trails[curr: curr + step]:
            task = asyncio.ensure_future(get_html_data(trail, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
        return responses


def process_df(future, curr, step):
    """Save scrape results in json files"""
    cache = {k: v for k, v in future.result()}
#     cache = future.result()
    if len(cache) == 0:
        raise RuntimeError("Empty response!")
    else:
#         json.dump(cache, open("./data/scraped/scraped_{0}_{1}.json".format(curr, curr + step), "w"))
        json.dump(cache, open("./data/mondo_scraped/contents_eco/scraped_{0}_{1}.json".format(curr, curr + step), "w"))
        print("got it! ({0}, {1})".format(curr, curr + step))

In [11]:
# df = pd.read_csv("./data/ratings.csv")
# app_ids = df.loc[:, "app_id"].astype(str)

start = 11350
# end = len(trails)
end = 11400
step = 50
DEBUG = []

for curr in range(start, end, step):
    print("loading data from {0} to {1}".format(curr, curr + step))
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(gather_results(curr, step, trails))
    future.add_done_callback(partial(process_df, curr=curr, step=step))
    loop.run_until_complete(future)

loading data from 11350 to 11400
got it! (11350, 11400)


## convert

In [12]:
doc_list = os.listdir("./data/mondo_scraped/contents_eco/")
articles = dict()
for file in doc_list:
    if file[-4:] == 'json':
        with open(os.path.join("./data/mondo_scraped/contents_eco/", file), "r") as f:
            articles.update(json.load(f))

len(articles)

14350

In [13]:
clean = {k: v for k, v in articles.items() if v is not None}

In [14]:
len(clean)

14328

In [15]:
docs = pd.DataFrame.from_dict(clean, orient='index')

In [16]:
valids = docs.loc[docs['keyfacts'].astype(np.bool), :]
valids.shape

(8031, 5)

In [17]:
valids.head()

,headline,keyfacts,content,tags,time
http://www.elmundo.es/america/2014/01/08/52cdabed268e3e892e8b458e.html,La Fed avisa de que no tiene marcada la senda ...,"[ Dependerá de la evolución de la economía, el...",La mayoría de los miembros del Comité Federal ...,[],2014-01-08
http://www.elmundo.es/america/2014/01/22/52dfe0eae2704ea74a8b4570.html,El Gobierno argentino sólo permitirá dos compr...,[ Cuando se supere el límite el comprador debe...,El Gobierno de Cristina Fernández de Kirchner ...,[],2014-01-22
http://www.elmundo.es/america/2014/01/26/52e52770e2704ecd598b4571.html,Marcha atrás en 48 horas del Gobierno argentin...,[ 'En la mentalidad argentina está insertado e...,"""Donde dije 'digo', digo 'Diego'"" parece procl...",[],2014-01-26
http://www.elmundo.es/america/2014/04/12/53492fd522601d410e8b456c.html,EEUU investiga a Herbalife por supuesta estafa...,[\nVende productos y suplementos dietéticos a ...,"La Comisión Federal del Comercio de EEUU (FTC,...",[],2014-04-12
http://www.elmundo.es/america/2014/04/21/535517cb268e3eb4218b457a.html,Argentina recurre al Supremo de EEUU contra lo...,[\nLos fondos compraron a precio de ganga bono...,Los ocho jueces del Tribunal Supremo de los Es...,[],2014-04-21


In [18]:
valids.to_pickle('./data/elmondo_es_eco.pkl')

In [19]:
valids.iloc[7, :].apply(print);

Multa de 5.000 millones a Citigroup por mentir a sus clientes
['\nEl banco ha anunciado su intención de recurrir a sus cuentas para hacer frente al pago ', '\nAcuerda pagar una multa de 5.140 millones de euros por la venta de hipotecas tóxicas ']
La multa de la quincena es para... Citigroup. Después de haber sancionado al suizo Credit Suisse con 1.910 millones de euros (2.600 millones de dólares) por ayudar a sus clientes a evadir impuestos, y al francés BNP Paribas con 6.540 millones de euros (8.900 millones de dólares) por ayudar a Sudán a evadir las sanciones impuestas por sus violaciones de los derechos humanos, EEUU ha apuntado en casa. 
Consecuencia: Citigroup, el tercer mayor banco del país por activos, ha sido multado con 5.140 millones de euros (7.000 millones de dólares) por haber engañado a sus clientes en la venta de hipotecas basura.
La entidad financiera, que ha dado a conocer el acuerdo, pagará algo más de 2.900 millones de euros en concepto de sanción civil al Departame